# Importing all the libraires needed for importing the model and procesing the image from the web-cam

In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import cv2
import numpy as np
from flask import request
from flask import Flask, Blueprint
import IPython

# Opening the web-cam, preprocessing the images retrived, makeing predictions for each frame and choseing the class with the most correct predictions¶

In [ ]:
Face_detector = Blueprint('Face_detector', __name__)

@Face_detector.route('/')
#Function that opens the camera and predict the face
def face_prediction():

    face_classifier =cv2.CascadeClassifier(r'C:/Users/Alexandru Neagu/Exercises/Team project/haarcascade_frontalface_default.xml')
    classifier= load_model(r'C:/Users/Alexandru Neagu/Exercises/Team project/Face_recognition/Checkpoint/Model-10.h5')

    class_labels=['Alexandru', 'Diana', 'Sebastian', 'Unknown']

    freq=[0,0,0,0]

    cap=cv2.VideoCapture(0)
    cap.set(3, 640) # set video widht
    cap.set(4, 480) # set video height

    minW = 0.1*cap.get(3)
    minH = 0.1*cap.get(4)

    count=0
    while True:
        # Read the frame
        _, img = cap.read()
        count+=1
        labels = []
        # Detect the faces
        faces = face_classifier.detectMultiScale( 
            img,
            scaleFactor = 1.2,
            minNeighbors = 5,
            minSize = (int(minW), int(minH)),
           )
        # Draw the rectangle around each face
        for (x, y, w, h) in faces:
            cv2.rectangle(img, (x, y), (x+w, y+h), (255, 0, 0), 2)
            roi_gray = img[y:y+h,x:x+w]
            
       
            #Get the image to the right dimension   
            cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray, (224, 224)), -1), 0)
            preds = classifier.predict(cropped_img)
            label=class_labels[np.argmax(preds)]
            #Createing a list with the freaquances of each prediction
            freq[class_labels.index(label)]+=1
            label_position = (x,y)
            cv2.putText(img,label,label_position,cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),3)
       
            print("\n\n")
        # Display
        cv2.imshow('img', img)
        # Stop if escape key is pressed
        k = cv2.waitKey(30) & 0xff
        if k==27:
            break
        elif count >= 20: # Take 20 face sample and stop video
            break
    # Release the VideoCapture object

    cap.release()
    cv2.destroyAllWindows()
    #Return the most predicted class
    return class_labels[freq.index(max(freq))]
    

def restartkernel() :
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel
    return "Kernal restarted"
    #Function that shuts down th
    e server
def shutdown_server():
    func = request.environ.get('werkzeug.server.shutdown')
    if func is None:
        raise RuntimeError('Not running with the Werkzeug Server')
    func()
    
@Face_detector.route('/restart', methods=['GET'])
def shutdown():
    shutdown_server()
    restartkernel()
    return 'Server shutting down...'

if __name__ == '__main__':
     
    #Running the appication on the server
    app = Flask(__name__)
    app.register_blueprint(Face_detector, url_prefix='/')
    app.run()
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


127.0.0.1 - - [19/Apr/2021 19:05:39] "GET / HTTP/1.1" 200 -
